<a href="https://colab.research.google.com/github/kamicosta/mercadolivreSBPO/blob/main/c%C3%B3digot%C3%B3picos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy matplotlib plotly scheptk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scheptk.scheptk import FlowShop
import time
import math
import random
import sys
import os
import pickle

# ... (Funções de apoio, heurísticas, VNS - sem alterações)
def calcular_metricas_sequencia(sequencia, p_matrix, num_maquinas):
    num_trabalhos = len(sequencia)
    if num_trabalhos == 0: return 0, 0
    C = np.zeros((num_trabalhos, num_maquinas))
    C[0, 0] = p_matrix[sequencia[0], 0]
    for m in range(1, num_maquinas): C[0, m] = C[0, m-1] + p_matrix[sequencia[0], m]
    for j_seq in range(1, num_trabalhos):
        C[j_seq, 0] = C[j_seq-1, 0] + p_matrix[sequencia[j_seq], 0]
        for m in range(1, num_maquinas):
            C[j_seq, m] = max(C[j_seq-1, m], C[j_seq, m-1]) + p_matrix[sequencia[j_seq], m]
    makespan = C[-1, -1]
    tempo_trabalhado = np.sum(p_matrix)
    idle_time_total = (num_maquinas * makespan) - tempo_trabalhado
    return makespan, idle_time_total

def calcular_makespan_sequencia(sequencia, p_matrix, num_maquinas):
    return calcular_metricas_sequencia(sequencia, p_matrix, num_maquinas)[0]

def calcular_limite_inferior(p_matrix):
    return max(np.max(np.sum(p_matrix, axis=0)), np.max(np.sum(p_matrix, axis=1)))

def heuristica_palmer(p_matrix, num_trabalhos, num_maquinas):
    indices = [(sum((num_maquinas - (2*i - 1)) * p_matrix[j, i-1] for i in range(1, num_maquinas + 1)), j) for j in range(num_trabalhos)]
    indices.sort(key=lambda x: x[0], reverse=True)
    return [idx for _, idx in indices]

def johnson_2_maquinas(p1, p2):
    n = len(p1); jobs = list(range(n))
    set1 = sorted([j for j in jobs if p1[j] < p2[j]], key=lambda j: p1[j])
    set2 = sorted([j for j in jobs if p1[j] >= p2[j]], key=lambda j: p2[j], reverse=True)
    return set1 + set2

def heuristica_cds(p_matrix, num_trabalhos, num_maquinas):
    melhor_sequencia, melhor_makespan = [], float('inf')
    for k in range(1, num_maquinas):
        p1, p2 = np.sum(p_matrix[:, :k], axis=1), np.sum(p_matrix[:, -k:], axis=1)
        sequencia_atual = johnson_2_maquinas(p1.tolist(), p2.tolist())
        makespan_atual = calcular_makespan_sequencia(sequencia_atual, p_matrix, num_maquinas)
        if makespan_atual < melhor_makespan: melhor_makespan, melhor_sequencia = makespan_atual, sequencia_atual
    return melhor_sequencia

def heuristica_gupta(p_matrix, num_trabalhos, num_maquinas):
    indices = []
    for j in range(num_trabalhos):
        e = 1 if p_matrix[j, 0] < p_matrix[j, num_maquinas - 1] else -1
        min_soma_adjacente = min(p_matrix[j, k] + p_matrix[j, k+1] for k in range(num_maquinas - 1))
        s_j = e / min_soma_adjacente if min_soma_adjacente > 0 else e * float('inf')
        indices.append((s_j, j))
    indices.sort(key=lambda x: x[0])
    return [idx for _, idx in indices]

def heuristica_neh_acelerada(p_matrix, num_trabalhos, num_maquinas):
    soma_tempos = np.sum(p_matrix, axis=1)
    trabalhos_ordenados = sorted(range(num_trabalhos), key=lambda k: soma_tempos[k], reverse=True)
    seq_parcial = [trabalhos_ordenados[0]]
    for i in range(1, num_trabalhos):
        job_a_inserir = trabalhos_ordenados[i]
        melhor_makespan, melhor_posicao = float('inf'), -1
        for k in range(i + 1):
            nova_seq = seq_parcial[:k] + [job_a_inserir] + seq_parcial[k:]
            makespan = calcular_makespan_sequencia(nova_seq, p_matrix, num_maquinas)
            if makespan < melhor_makespan: melhor_makespan, melhor_posicao = makespan, k
        seq_parcial.insert(melhor_posicao, job_a_inserir)
    return seq_parcial

def busca_local_vns_acelerada(sequencia, p_matrix, num_maquinas):
    melhor_sequencia = sequencia[:]; melhor_makespan, melhor_idle = calcular_metricas_sequencia(melhor_sequencia, p_matrix, num_maquinas)
    vizinhancas = [explorar_vizinhanca_insertion_acelerada, explorar_vizinhanca_swap, explorar_vizinhanca_2opt]
    i_vizinhanca = 0
    while i_vizinhanca < len(vizinhancas):
        nova_sequencia, novo_makespan, novo_idle, houve_melhora = vizinhancas[i_vizinhanca](melhor_sequencia, p_matrix, num_maquinas, melhor_makespan, melhor_idle)
        if houve_melhora:
            melhor_sequencia, melhor_makespan, melhor_idle = nova_sequencia, novo_makespan, novo_idle
            i_vizinhanca = 0
        else:
            i_vizinhanca += 1
    return melhor_sequencia, melhor_makespan

def explorar_vizinhanca_swap(seq, p_matrix, num_maquinas, atual_best_m, atual_best_i):
    for i in range(len(seq) - 1):
        for j in range(i + 1, len(seq)):
            nova_seq = seq[:]; nova_seq[i], nova_seq[j] = nova_seq[j], nova_seq[i]
            novo_makespan, novo_idle = calcular_metricas_sequencia(nova_seq, p_matrix, num_maquinas)
            if novo_makespan < atual_best_m or (novo_makespan == atual_best_m and novo_idle < atual_best_i): return nova_seq, novo_makespan, novo_idle, True
    return seq, atual_best_m, atual_best_i, False

def explorar_vizinhanca_insertion_acelerada(seq, p_matrix, num_maquinas, atual_best_m, atual_best_i):
    for i in range(len(seq)):
        job_a_mover = seq[i]; seq_parcial = seq[:i] + seq[i+1:]
        c_matrix = calcular_c_matrix(seq_parcial, p_matrix, num_maquinas); q_matrix = calcular_q_matrix(seq_parcial, p_matrix, num_maquinas)
        for k in range(len(seq)):
            if i == k: continue
            c_job = np.zeros(num_maquinas); prev_c_job = c_matrix[k-1] if k > 0 else np.zeros(num_maquinas)
            c_job[0] = prev_c_job[0] + p_matrix[job_a_mover, 0]
            for m in range(1, num_maquinas): c_job[m] = max(c_job[m-1], prev_c_job[m]) + p_matrix[job_a_mover, m]
            next_q_job = q_matrix[k] if k < len(seq_parcial) else np.zeros(num_maquinas)
            makespan_teste = np.max(c_job + next_q_job)
            if makespan_teste < atual_best_m:
                nova_seq = seq_parcial[:k] + [job_a_mover] + seq_parcial[k:]
                novo_makespan, novo_idle = calcular_metricas_sequencia(nova_seq, p_matrix, num_maquinas)
                return nova_seq, novo_makespan, novo_idle, True
            elif makespan_teste == atual_best_m:
                nova_seq = seq_parcial[:k] + [job_a_mover] + seq_parcial[k:]
                _, novo_idle = calcular_metricas_sequencia(nova_seq, p_matrix, num_maquinas)
                if novo_idle < atual_best_i: return nova_seq, makespan_teste, novo_idle, True
    return seq, atual_best_m, atual_best_i, False

def explorar_vizinhanca_2opt(seq, p_matrix, num_maquinas, atual_best_m, atual_best_i):
    for i in range(len(seq) - 1):
        for j in range(i + 2, len(seq) + 1):
            nova_seq = seq[:i] + seq[i:j][::-1] + seq[j:]
            novo_makespan, novo_idle = calcular_metricas_sequencia(nova_seq, p_matrix, num_maquinas)
            if novo_makespan < atual_best_m or (novo_makespan == atual_best_m and novo_idle < atual_best_i): return nova_seq, novo_makespan, novo_idle, True
    return seq, atual_best_m, atual_best_i, False

def calcular_c_matrix(seq, p_matrix, num_maquinas):
    c = np.zeros((len(seq), num_maquinas));
    if not seq: return c
    c[0, 0] = p_matrix[seq[0], 0]
    for m in range(1, num_maquinas): c[0, m] = c[0, m-1] + p_matrix[seq[0], m]
    for j_seq in range(1, len(seq)):
        c[j_seq, 0] = c[j_seq-1, 0] + p_matrix[seq[j_seq], 0]
        for m in range(1, num_maquinas): c[j_seq, m] = max(c[j_seq-1, m], c[j_seq, m-1]) + p_matrix[seq[j_seq], m]
    return c

def calcular_q_matrix(seq, p_matrix, num_maquinas):
    q = np.zeros((len(seq), num_maquinas)); n_seq = len(seq)
    if not seq: return q
    q[n_seq-1, num_maquinas-1] = p_matrix[seq[n_seq-1], num_maquinas-1]
    for m in range(num_maquinas-2, -1, -1): q[n_seq-1, m] = q[n_seq-1, m+1] + p_matrix[seq[n_seq-1], m]
    for j_seq in range(n_seq-2, -1, -1):
        q[j_seq, num_maquinas-1] = q[j_seq+1, num_maquinas-1] + p_matrix[seq[j_seq], num_maquinas-1]
        for m in range(num_maquinas-2, -1, -1): q[j_seq, m] = max(q[j_seq+1, m], q[j_seq, m+1]) + p_matrix[seq[j_seq], m]
    return q

# =============================================================================
# <<< VERSÃO CORRIGIDA DA FUNÇÃO DE GANTT >>>
# =============================================================================
# =============================================================================
# <<< VERSÃO CORRIGIDA DA FUNÇÃO DE GANTT (CORES MAIS VIBRANTES) >>>
# =============================================================================
def gerar_grafico_gantt(sequencia, p_matrix, num_maquinas, makespan):
    num_trabalhos = len(sequencia)
    C = np.zeros((num_trabalhos, num_maquinas))
    S = np.zeros((num_trabalhos, num_maquinas))
    for j_seq, job_idx in enumerate(sequencia):
        for m in range(num_maquinas):
            S[j_seq, m] = max(C[j_seq, m-1] if m > 0 else 0, C[j_seq-1, m] if j_seq > 0 else 0)
            C[j_seq, m] = S[j_seq, m] + p_matrix[job_idx, m]

    fig, ax = plt.subplots(figsize=(18, 7))
    # <<< ALTERADO: Paleta de cores 'hsv' para maior variedade de tons.
    cores = plt.cm.hsv(np.linspace(0, 1, num_trabalhos + 1))

    yticks = []
    yticklabels = []
    altura = 0.8

    for m in range(num_maquinas):
        for j_seq, job_idx in enumerate(sequencia):
            start = S[j_seq, m]
            dur = p_matrix[job_idx, m]
            rect = patches.Rectangle(
                (start, m - altura/2), dur, altura,
                facecolor=cores[job_idx % len(cores)],
                edgecolor='black', linewidth=1.2, alpha=0.95
            )
            ax.add_patch(rect)
            # Label do job
            if dur > 0.04 * makespan or num_trabalhos <= 30:
                ax.text(
                    start + dur/2, m,
                    f'T{job_idx+1}',
                    ha='center', va='center',
                    color='white' if dur > 0.1 * makespan and num_trabalhos < 50 else 'black',
                    fontsize=10, fontweight='bold'
                )
        yticks.append(m)
        yticklabels.append(f'Máquina {m+1}')

    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels, fontsize=12)
    ax.set_xlabel('Tempo', fontsize=13, weight='bold')
    ax.set_ylabel('Recursos', fontsize=13, weight='bold')
    ax.set_title(f'Gráfico de Gantt da Melhor Solução\nMakespan = {makespan:.0f}', fontsize=18, weight='bold', pad=20)
    ax.grid(axis='x', linestyle=':', color='gray', alpha=0.7)
    ax.set_xlim(0, makespan * 1.03)
    ax.set_ylim(-0.7, num_maquinas - 0.3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.2)
    ax.spines['bottom'].set_linewidth(1.2)

    plt.tight_layout()
    plt.show()

# =============================================================================
# MOTOR HÍBRIDO (VERSÃO COMPETIÇÃO COM INTELIGÊNCIA ADAPTATIVA)
# =============================================================================
CHECKPOINT_FILE = 'checkpoint.pkl'
PAUSE_FILE = 'pause.flag'

def salvar_estado(estado):
    with open(CHECKPOINT_FILE, 'wb') as f:
        pickle.dump(estado, f)

def carregar_estado():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'rb') as f: return pickle.load(f)
    return None

def motor_hibrido_iga_adaptativo(estado, p_matrix, num_maquinas, d_inicial, temperatura_base, trabalhos_criticos, start_time, limite_tempo):
    # Desempacota o estado
    seq_atual, makespan_atual = estado['seq_atual'], estado['makespan_atual']
    melhor_seq_global, makespan_global = estado['melhor_seq_global'], estado['makespan_global']
    iteracao, iter_sem_melhora_global = estado['iteracao'], estado.get('iter_sem_melhora_global', 0)
    d = estado.get('d', d_inicial)
    random.setstate(estado['random_state'])

    print(f"Continuando busca da iteração {iteracao} com makespan base = {makespan_global:.2f}. Pressione Ctrl+C para interromper.")

    try:
        while True:
            # Verificação do timer a cada iteração
            if limite_tempo is not None and (time.time() - start_time) >= limite_tempo:
                print("\n\n⏳ Tempo limite atingido. Finalizando busca...")
                break

            iteracao += 1
            iter_sem_melhora_global += 1

            if iter_sem_melhora_global > 150 and d < num_trabalhos // 2:
                d += 1
                iter_sem_melhora_global = 0
                print(f"\r[Iteração {iteracao}] Estagnação detectada. Aumentando destruição para d={d}...", end="")

            seq_a_destruir = seq_atual[:]
            trabalhos_removidos = []
            if random.random() < 0.7 and trabalhos_criticos:
                job_critico = random.choice(trabalhos_criticos)
                if job_critico in seq_a_destruir:
                    seq_a_destruir.remove(job_critico); trabalhos_removidos.append(job_critico)
            while len(trabalhos_removidos) < d and seq_a_destruir:
                trabalhos_removidos.append(seq_a_destruir.pop(random.randrange(len(seq_a_destruir))))

            seq_reconstruida = seq_a_destruir
            for job in trabalhos_removidos:
                min_makespan_parcial, melhor_pos_insercao = float('inf'), -1
                for i in range(len(seq_reconstruida) + 1):
                    makespan_parcial = calcular_makespan_sequencia(seq_reconstruida[:i] + [job] + seq_reconstruida[i:], p_matrix, num_maquinas)
                    if makespan_parcial < min_makespan_parcial:
                        min_makespan_parcial, melhor_pos_insercao = makespan_parcial, i
                seq_reconstruida.insert(melhor_pos_insercao, job)

            nova_seq, novo_makespan = busca_local_vns_acelerada(seq_reconstruida, p_matrix, num_maquinas)

            temperatura = temperatura_base / math.log(iteracao + 1)
            if novo_makespan < makespan_atual or random.random() < math.exp(-(novo_makespan - makespan_atual) / temperatura):
                seq_atual, makespan_atual = nova_seq[:], novo_makespan
                if makespan_atual < makespan_global:
                    makespan_global, melhor_seq_global = makespan_atual, nova_seq[:]
                    iter_sem_melhora_global = 0
                    d = d_inicial
                    print(f"\r[Iteração {iteracao}] 🎉 NOVO MELHOR GLOBAL: {makespan_global:.2f} (com d={d})", end="")

            if iteracao % 20 == 0:
                print(f"\r[Iteração {iteracao}] Buscando... Melhor: {makespan_global:.2f} | Atual: {makespan_atual:.2f} | d: {d}", end="")
                if os.path.exists(PAUSE_FILE):
                    print("\n\n⏸️ Pausa solicitada. Salvando checkpoint...")
                    estado_atual = {'seq_atual': seq_atual, 'makespan_atual': makespan_atual,
                                    'melhor_seq_global': melhor_seq_global, 'makespan_global': makespan_global,
                                    'iteracao': iteracao, 'random_state': random.getstate(),
                                    'iter_sem_melhora_global': iter_sem_melhora_global, 'd': d}
                    salvar_estado(estado_atual); os.remove(PAUSE_FILE)
                    print("Checkpoint salvo."); sys.exit()

    except KeyboardInterrupt: print("\n\n🛑 Busca interrompida.")
    return melhor_seq_global, makespan_global

# =============================================================================
# BLOCO DE EXECUÇÃO PRINCIPAL
# =============================================================================
if __name__ == '__main__':
    # Adapte o caminho do arquivo para a sua máquina
    Instâncias = '/content/drive/MyDrive/instancias/instanciasp/P19.txt'
    try:
        instance = FlowShop(Instâncias); num_trabalhos, num_maquinas = instance.jobs, instance.machines
        p_matrix = np.array(instance.pt).T
        print(f"✅ Instância {os.path.basename(Instâncias)}: {num_trabalhos} trabalhos, {num_maquinas} máquinas.")
    except Exception as e: print(f"❌ ERRO CRÍTICO: Não foi possível ler o arquivo. Verifique o caminho.\n{e}"); sys.exit()

    # Bloco para perguntar e validar o tempo de execução
    limite_tempo_segundos = None
    while True:
        try:
            resposta = input("Defina o tempo limite de execução em segundos (ex: 60). Deixe em branco ou 0 para rodar indefinidamente: ")
            if resposta.strip() == "" or int(resposta) == 0:
                print("OK, o otimizador rodará sem limite de tempo (até ser interrompido com Ctrl+C).")
                limite_tempo_segundos = None
                break
            limite_tempo_segundos = int(resposta)
            if limite_tempo_segundos > 0:
                print(f"OK, o otimizador rodará por {limite_tempo_segundos} segundos.")
                break
            else:
                print("Por favor, insira um número positivo.")
        except ValueError:
            print("Entrada inválida. Por favor, insira um número inteiro.")

    estado_salvo = carregar_estado()

    if estado_salvo:
        print("\n--- CONTINUANDO BUSCA A PARTIR DE CHECKPOINT ---")
        estado_para_motor = estado_salvo
    else:
        print("\n--- ESTÁGIO 1: BUSCANDO PONTO DE PARTIDA DE ELITE (Multi-Start VNS Acelerado) ---")
        sementes = {'Palmer': heuristica_palmer(p_matrix, num_trabalhos, num_maquinas),
                    'CDS': heuristica_cds(p_matrix, num_trabalhos, num_maquinas),
                    'Gupta': heuristica_gupta(p_matrix, num_trabalhos, num_maquinas),
                    'NEH': heuristica_neh_acelerada(p_matrix, num_trabalhos, num_maquinas)}

        campeao_inicial = (None, float('inf')); start_total = time.time()
        for nome, seq_inicial in sementes.items():
            seq_vns, makespan_vns = busca_local_vns_acelerada(seq_inicial, p_matrix, num_maquinas)
            if makespan_vns < campeao_inicial[1]: campeao_inicial = (seq_vns, makespan_vns)
        print(f"   - Concluído em {time.time() - start_total:.2f}s.")
        print(f"🏆 Ponto de Partida de Elite Encontrado: Makespan = {campeao_inicial[1]:.2f}")

        estado_para_motor = {'seq_atual': campeao_inicial[0], 'makespan_atual': campeao_inicial[1],
                             'melhor_seq_global': campeao_inicial[0], 'makespan_global': campeao_inicial[1],
                             'iteracao': 0, 'random_state': random.getstate()}

    print("\n--- ESTÁGIO 2: LANÇANDO MOTOR HÍBRIDO COM INTELIGÊNCIA ADAPTATIVA ---")

    # Parâmetros
    soma_tempos = np.sum(p_matrix, axis=1)
    trabalhos_criticos = sorted(range(num_trabalhos), key=lambda k: soma_tempos[k], reverse=True)[:max(5, int(num_trabalhos*0.25))]
    d_inicial = max(2, int(num_trabalhos * 0.15))
    temperatura_base = estado_para_motor['makespan_global'] * 0.05

    # Captura o tempo de início antes de chamar o motor
    start_time_execucao = time.time()

    # Passa os novos parâmetros para a função do motor
    seq_final, makespan_final = motor_hibrido_iga_adaptativo(
        estado_para_motor, p_matrix, num_maquinas, d_inicial, temperatura_base,
        trabalhos_criticos, start_time=start_time_execucao, limite_tempo=limite_tempo_segundos
    )

    print("\n\n--- 👑 RESULTADO FINAL DA COMPETIÇÃO 👑 ---")
    limite_inferior = calcular_limite_inferior(p_matrix)
    print(f"Melhor Sequência Encontrada: {[job + 1 for job in seq_final]}")
    print(f"Makespan Final (Cmax): {makespan_final:.2f}")
    print(f"Limite Inferior (LB):      {limite_inferior:.2f}")
    gap = ((makespan_final - limite_inferior) / limite_inferior) * 100 if limite_inferior > 0 else 0
    print(f"GAP percentual em relação ao LB: {gap:.2f}%")

    if os.path.exists(CHECKPOINT_FILE): os.remove(CHECKPOINT_FILE)
    gerar_grafico_gantt(seq_final, p_matrix, num_maquinas, makespan_final)